In [1]:
hostname = 'localhost'
username = 'root'
password = ''
database = 'chatbot'

import pymysql
myConnection = pymysql.connect( host=hostname, user=username, passwd=password, db=database )

In [14]:
def insertuser( conn ) :
    cur = conn.cursor()
    
    today = datetime.now()
#     print(today)
    dt_string = today.strftime("%Y/%m/%d %H:%M:%S")
    print(dt_string)
    cur.execute( "INSERT into user (session_time) values ('{}')".format(dt_string))
    conn.commit()
    print(cur.lastrowid)
    return int(cur.lastrowid)

def generate_questions(conn,emo):
    cur = conn.cursor()
    cur.execute( "SELECT * FROM question_list" )
    li = []
    for qid,question in cur.fetchall():
        print(qid,question)
        li.append([qid,question.format(emo)])
    return li

def updateUserFeeling( conn,feeling,uid ) :
    cur = conn.cursor()
    cur.execute( "UPDATE user SET feeling = '{}' where id = '{}'".format(feeling,uid))
    conn.commit()
    print(cur.rowcount)
    
def updateUserName( conn,name,uid ) :
    cur = conn.cursor()
    cur.execute( "UPDATE user SET name = '{}' where id = '{}'".format(name,uid))
    conn.commit()
    print(cur.rowcount)
    
def insertResponse( conn,uid,qid,response ) :
    cur = conn.cursor()
    cur.execute( "INSERT into user_responses (user_id,q_id,response) values ('{}','{}','{}')".format(uid,qid,response))
    conn.commit()
    print(cur.lastrowid)

In [ ]:

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from datetime import datetime

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

GREETING = 1
ASK_FEELING = 2
OPEN_ENDED = 3
ASK_QUESTION = 4
SURVEY = 5
FINISH = 6
ASK_NAME = 7
state = GREETING

EMOTION = ""
USER_ID = -1

questions = []
index = 0
response_survey = {}




def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    print(ints)
    res = getResponse(ints, intents)
    print(res)
    tag = ints[0]['intent']
    global state
    global questions
    global USER_ID
    global EMOTION
    if state==GREETING:
        uid = insertuser(myConnection)
        USER_ID = uid
        state=ASK_NAME
    elif state == ASK_NAME:
        state = ASK_FEELING
        
    elif state==ASK_FEELING:
        state=OPEN_ENDED
        EMOTION = EMOTION + ints[0]['intent']
        print(EMOTION,USER_ID)
        updateUserFeeling(myConnection,EMOTION,USER_ID)
        print(EMOTION)
        questions = questions + generate_questions(myConnection,EMOTION)
        print(questions)
    elif state==OPEN_ENDED:
        state = ASK_QUESTION
    elif state == ASK_QUESTION and tag=='yes':
        state=SURVEY
    elif state == ASK_QUESTION and tag=='no':
        state=FINISH
        
    
    return res,tag


#Creating GUI with tkinter
import tkinter
from tkinter import *

def send_response(response):
    ChatLog.insert(END, "Bot: " + response + '\n\n')

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    global state
    global index
    global response_survey
    global USER_ID
    if msg != '' and state!=SURVEY and state!=ASK_NAME:
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Arial", 12 ))

        res,tag = chatbot_response(msg)
        ChatLog.insert(END, "Raah: " + res + '\n\n')
        if state==SURVEY and tag=='yes':
            send_response('For the next 5 questions, enter the number corresponding to each response')
            ChatLog.insert(END, "Q{}: ".format(index+1) + questions[index][1] + '\n\n')
            index+=1
            ChatLog.insert(END, "1: Stongly Agree\n")
            ChatLog.insert(END, "2: Agree\n")
            ChatLog.insert(END, "3: Neither Agree nor Disagree\n")
            ChatLog.insert(END, "4: Disagree\n")
            ChatLog.insert(END, "5: Stongly Disagree\n")
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    elif msg != '' and state==SURVEY:
        ChatLog.config(state=NORMAL)
        valid  = ['1','2','3','4','5']
        if msg in valid :
            ChatLog.insert(END, "You: " + msg + '\n\n')
            response_survey[questions[index-1][1]] = msg
            insertResponse( myConnection,USER_ID,questions[index-1][0],msg)
            if index<5:
                ChatLog.insert(END, "Q{}: ".format(index+1) + questions[index][1] + '\n\n')
                index+=1
                ChatLog.insert(END, "1: Stongly Agree\n")
                ChatLog.insert(END, "2: Agree\n")
                ChatLog.insert(END, "3: Neither Agree nor Disagree\n")
                ChatLog.insert(END, "4: Disagree\n")
                ChatLog.insert(END, "5: Stongly Disagree\n\n")
            else: 
                ChatLog.insert(END, "Raah: Thank You For Answering, Have a good day :)\n\n")
                print('check',EMOTION)
                if EMOTION != 'good':
                    ChatLog.insert(END, "Raah: Just Remember, It’s Completely Normal To Feel This Way! ")
                state = FINISH
                
        else:
            ChatLog.insert(END, "Raah: Invalid Response, Try Again\n\n")
        
        ChatLog.config(foreground="#442265", font=("Arial", 12 ))
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    elif msg != '' and state == ASK_NAME:
        state = ASK_FEELING
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        updateUserName(myConnection,msg,USER_ID)
        ChatLog.config(foreground="#442265", font=("Arial", 12 ))
        # ASK NAME
        ChatLog.insert(END, 'Raah: Hi {}, How are you feeling today? \nGood, Bad or Neutral?\n\n'.format(msg))
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)



        
        


base = Tk()
base.title("TheOther 2 Thirds")
base.geometry("400x500")
base.resizable(width=TRUE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


[{'intent': 'greeting', 'probability': '0.99997234'}]
Hi, I'm Raah! What's your name?
2020/11/14 12:56:01
52
1
[{'intent': 'good', 'probability': '0.8811887'}]
That's Wonderful!  What makes you happy?
good 52
1
good
12 Are you feeling {} because of your work related satisfaction?
13 Are you feeling {} because of reasons outside your work?
14 Do You feel {} because of monetory reasons?
15 Are you feeling {} because of work-life balance?
16 Are you feeling {} because of your interactions with coworkers or superiors?
[[12, 'Are you feeling good because of your work related satisfaction?'], [13, 'Are you feeling good because of reasons outside your work?'], [14, 'Do You feel good because of monetory reasons?'], [15, 'Are you feeling good because of work-life balance?'], [16, 'Are you feeling good because of your interactions with coworkers or superiors?']]
[{'intent': 'openended', 'probability': '0.99981266'}]
Oh! Well, if you don't mind, may I ask a few questions?
[{'intent': 'openended',

In [29]:
print(response_survey)

{'You are feeling bad because of your work environment': '1', 'You are feeling bad because of reason outside your work': '2', 'You feel bad because of monetory reasons': '4', 'You are feeling bad because of work-life balance': '2'}


In [17]:
print(insertuser(myConnection))

2020/11/14 12:55:32
51
51


In [29]:
updateUserFeeling(myConnection,'bad',9)

1


In [6]:
def insertQuestions( conn ) :
    cur = conn.cursor()
    global li
    for q in li:
        cur.execute( "INSERT into question_list (question) values ('{}')".format(q))
        conn.commit()
        print(cur.lastrowid)

In [7]:
insertQuestions(myConnection)

12
13
14
15
16


In [3]:
def generate_questions_to_insert():
    li = []
    li.append('Are you feeling {} because of your work related satisfaction?')
    li.append('Are you feeling {} because of reasons outside your work?')
    li.append('Do You feel {} because of monetory reasons?')
    li.append('Are you feeling {} because of work-life balance?')
    li.append('Are you feeling {} because of your interactions with coworkers or superiors?')
    return li

In [4]:
li = generate_questions_to_insert()

In [54]:
generate_questions('bad',myConnection)

7 You are feeling {} because of your work environment
8 You are feeling {} because of reason outside your work
9 You feel {} because of monetory reasons
10 You are feeling {} because of work-life balance
11 You are feeling {} because of your interactions with coworkers


[[7, 'You are feeling bad because of your work environment'],
 [8, 'You are feeling bad because of reason outside your work'],
 [9, 'You feel bad because of monetory reasons'],
 [10, 'You are feeling bad because of work-life balance'],
 [11, 'You are feeling bad because of your interactions with coworkers']]

In [14]:
print('😊')

😊
